In [ ]:
!pip install rouge

In [ ]:
glob('../input/pl-articles-split/pl_spacy_model-0.1.0/*')

In [ ]:
!python -m pip install ../input/pl-articles-split/pl_spacy_model-0.1.0/pl_spacy_model-0.1.0

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import random
import re
import spacy
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.nn.utils import clip_grad_norm_

from glob import glob
from spacy.symbols import LEMMA, ORTH, POS
from sklearn.model_selection import train_test_split
from tqdm import notebook, tqdm_notebook
from torchtext.data import Field, BucketIterator, TabularDataset

from rouge import Rouge

## Load data

In [ ]:
glob('../input/*/*')

In [ ]:
nlp = spacy.load('en', disable=['ner', 'parser'])

In [ ]:
special_tokens = ['<num>']

In [ ]:
special_case = [{POS: 'NOUN', ORTH: '<num>', LEMMA: '<num>'}]
nlp.tokenizer.add_special_case("<num>", special_case)

In [ ]:
def tokenize_pl(text):
    text = re.sub(r"(\„|\”|\“|\‟|\‶|\‚|\’|\‘|\‛|\⁏|\;|\-|\—|\―|\–|\⁋|\‰|\\|\%|\^|\&|\*|\$|\#|\@|\!)", '', str(text))
    text = re.sub("\.\.+", ' ', str(text))
#     text = re.sub("\.", ' .', str(text))
    text = re.sub(r"(\/)", ' ', str(text))
    text = re.sub("[0-9]+", " <num> ", str(text)) # hide numbers
#     text = re.sub("[0-9]+", ' ', str(text))
    return [tok.text for tok in nlp.tokenizer(re.sub("\s+", ' ', str(text)))]

## Get dataset

In [ ]:
def load_dataset(batch_size, special_tokens=None):

    TEXT = Field(tokenize=tokenize_pl, include_lengths=True, tokenizer_language='pl',
                init_token='<sos>', eos_token='<eos>')
    SUMMARY = Field(tokenize=tokenize_pl, include_lengths=True, tokenizer_language='pl',
                    init_token='<sos>', eos_token='<eos>')
    train, val, test = TabularDataset.splits(
        skip_header=True, 
        path='../input/pl-articles-split/', format='csv', 
        fields=[('index', None), ('lead', SUMMARY), ('text', TEXT)],
        train='train.csv', validation='val.csv', test='test.csv'
    )
    TEXT.build_vocab(train, specials=special_tokens, min_freq=10)
#     SUMMARY.build_vocab(train, min_freq=2)
    SUMMARY.vocab = TEXT.vocab
    train_iter, val_iter, test_iter = BucketIterator.splits(
        (train, val, test), batch_size=batch_size, repeat=False, sort_key=lambda x: len(x.text), sort_within_batch=False
    )
    return train_iter, val_iter, test_iter, TEXT, SUMMARY, (train, val, test)

In [ ]:
batch_size = 16

In [ ]:
train_iter, val_iter, test_iter, TEXT, SUMMARY, _ = load_dataset(batch_size, special_tokens)

In [ ]:
def indices_from_text(text, lang=TEXT):
    indices = []
    for word in text.strip().split(' '):
        indices.append(lang.vocab.stoi[word])
    return torch.LongTensor(indices).cuda()

In [ ]:
def text_from_indices(indices, lang=TEXT):
    text = ""
    for element in indices:
        if type(element) is torch.Tensor:
            text += lang.vocab.itos[element.item()] + " "
        else:
            text += lang.vocab.itos[element] + " "
    return text

In [ ]:
len(TEXT.vocab.stoi)

In [ ]:
batch = next(iter(train_iter))

#### Random text

In [ ]:
text_from_indices(batch.text[0].transpose(0, 1)[-1])

#### Random summary

In [ ]:
text_from_indices(batch.lead[0].transpose(0, 1)[-1])

## Seq2Seq model

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, n_layers=1, dropout=0.1):
        super(EncoderRNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.embedding = nn.Embedding(input_size, embedding_size).cuda()
        self.gru = nn.GRU(embedding_size, hidden_size, n_layers,
                          dropout=dropout, bidirectional=True).cuda()

    def forward(self, sequence, hidden=None):
        embedding_output = self.embedding(sequence) # max_text_len x batch_size x embedding_size
        encoder_outputs, hidden = self.gru(embedding_output, hidden)
        # hidden: bidirectional x batch_size x hidden_size
        # output: max_text_len x batch_size x bidirectional * hidden_size
        encoder_outputs = encoder_outputs[:, :, :self.hidden_size] + encoder_outputs[:, :, self.hidden_size:]
        # output: max_text_len x batch_size x hidden_size
        encoder_outputs = encoder_outputs
        return encoder_outputs, hidden

In [ ]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.hidden_size = hidden_size
        self.attention = nn.Linear(hidden_size * 2, hidden_size).cuda()
        self.v = nn.Parameter(torch.rand(hidden_size)).cuda()
        stdv = 1. / math.sqrt(self.v.size(0))
        self.v.data.uniform_(-stdv, stdv)

    def forward(self, hidden, encoder_outputs):
        timestep = encoder_outputs.size(0)
        h = hidden.repeat(timestep, 1, 1).transpose(0, 1)
        encoder_outputs = encoder_outputs.transpose(0, 1) 
        attn_energies = self.score(h, encoder_outputs) # batch_size x t x hidden
        return F.softmax(attn_energies, dim=1).unsqueeze(1) # batch_size x t

    def score(self, hidden, encoder_outputs):
        # batch_size x t x 2*hidden -> batch_size x t x hidden
        energy = torch.tanh(self.attention(torch.cat([hidden, encoder_outputs], 2)))
        energy = energy.transpose(1, 2) # batch_size x t x 2*hidden -> batch_size x t x hidden
        v = self.v.repeat(encoder_outputs.size(0), 1).unsqueeze(1) # batch_size x 1 x hidden
        energy = torch.bmm(v, energy) # batch_size x 1 x t
        return energy.squeeze(1) # batch_size x t

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, embedding_size, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(DecoderRNN, self).__init__()
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        self.embedding = nn.Embedding(output_size, embedding_size).cuda()
        self.dropout = nn.Dropout(dropout, inplace=True).cuda()
        self.attention = BahdanauAttention(hidden_size).cuda()
        self.gru = nn.GRU(hidden_size + embedding_size, hidden_size, n_layers, dropout=dropout).cuda()
        self.output = nn.Linear(hidden_size * 2, output_size).cuda()

    def forward(self, sequence, hidden, encoder_outputs):
        # Get the embedding of the current input word (last output word)
        embedding_output = self.embedding(sequence).unsqueeze(0)  # 1 x batch_size x n
        embedding_output = self.dropout(embedding_output)
        # Calculate attention weights and apply to encoder outputs
        attention_weights = self.attention(hidden[-1], encoder_outputs)
        context = attention_weights.bmm(encoder_outputs.transpose(0, 1)) # batch_size x 1 x n
        context = context.transpose(0, 1)  # (1,B,N)
        # Combine embedded input word and attended context, run through RNN
        decoder_input = torch.cat([embedding_output, context], 2)
        decoder_output, hidden = self.gru(decoder_input, hidden)
        decoder_output = decoder_output.squeeze(0)  # (1,B,N) -> (B,N)
        context = context.squeeze(0)
        decoder_output = self.output(torch.cat([decoder_output, context], 1))
        decoder_output = F.log_softmax(decoder_output, dim=1)
        return decoder_output, hidden, attention_weights

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, text, summary, teacher_forcing_ratio=0.5):
        batch_size = text.size(1)
        max_len = summary.size(0)
        vocab_size = self.decoder.output_size

        encoder_output, hidden = self.encoder(text)
        hidden = hidden[:self.decoder.n_layers]
        output = summary.data[0, :]  # sos
        
        outputs = torch.cuda.FloatTensor(max_len, batch_size, vocab_size).fill_(0)
        for t in range(1, max_len):
            output, hidden, attention_weights = self.decoder(
                    output, hidden, encoder_output)
            outputs[t] = output
            is_teacher = random.random() < teacher_forcing_ratio
            top_first = output.data.max(1)[1]
            output = summary.data[t] if is_teacher else top_first
        return outputs

In [ ]:
rouge = Rouge()

## Train

In [ ]:
def train(e, model, optimizer, scheduler, train_iter, vocab_size, grad_clip, lang=TEXT):
    model.train()
    total_loss = 0
    pad = lang.vocab.stoi['<pad>']
#     train_iter = train_iter.cuda()
    for b, batch in notebook.tqdm(enumerate(train_iter), total=len(train_iter)):
        text, len_text = batch.text
        summary, len_summary = batch.lead
        text, summary = text.cuda(), summary.cuda()
        optimizer.zero_grad()
        output = model(text, summary)
        loss = F.nll_loss(
            output[1:].view(-1, vocab_size),
            summary[1:].contiguous().view(-1),
            ignore_index=pad,
        )
        loss.backward()
        clip_grad_norm_(model.parameters(), grad_clip)
        optimizer.step()
        scheduler.step()
        total_loss += loss.data
        
        del output
        del loss
        
        if b % 10 == 0 and b != 0:
            total_loss = total_loss / 100
            print(f'[{b} / {len(train_iter)}]  [loss: {total_loss}]')
            total_loss = 0
            torch.cuda.empty_cache()
        if b % 400 == 0 and b != 0:
            original_text = text_from_indices(text.transpose(0, 1)[0])
            target_summary = text_from_indices(summary.transpose(0, 1)[0])
            output_summary, attentions = summarize(text_from_indices(text.transpose(0, 1)[0]))
            print('Original :', original_text)
            print('-' * 80)
            print('Target :', target_summary)
            print('-' * 80)
            print('Summary :', output_summary)
            print('=' * 80)
            scores = calculate_rouge(hypothesis=output_summary, reference=target_summary)
            if scores:
                for key, value in scores[0].items():
                    print(f'{key.upper()} [precision] : {np.round(value["p"] * 100, 2)} '
                          f'| [recall] : {np.round(value["r"] * 100, 2)} '
                          f'| [f-score] : {np.round(value["f"] * 100, 2)}')
                draw_attention_matrix(attention=attentions, original=original_text, summary=output_summary)

In [ ]:
def evaluate(model, val_iter, vocab_size, lang=TEXT):
#     val_iter = val_iter.cuda()
    with torch.no_grad():
        pad = lang.vocab.stoi['<pad>']
        total_loss = 0
        for b, batch in enumerate(val_iter):
            text, len_text = batch.text
            summary, len_summary = batch.lead
#             text = text.data
#             summary = summary.data
            text = text.data.cuda()
            summary = summary.data.cuda()
            output = model(text, summary, teacher_forcing_ratio=0.0)
            loss = F.nll_loss(
                output[1:].view(-1, vocab_size),
                summary[1:].contiguous().view(-1),
                ignore_index=pad,
            )
            total_loss += loss.data
        return total_loss / len(val_iter)

In [ ]:
def calculate_rouge(hypothesis, reference):
    hypothesis = hypothesis.split('<sos>')[1].split('<eos>')[0].strip()
    reference = reference.split('<sos>')[1].split('<eos>')[0].strip()
    if len(hypothesis) > 0:
        scores = rouge.get_scores(hypothesis, reference)
        return scores
    return None

In [ ]:
def draw_attention_matrix(attention, original, summary):
    labels_original = original.split('<sos>')[1].split('<eos>')[0].strip().split()
    labels_summary = summary.split('<sos>')[1].split('<eos>')[0].strip().split()
    plt.figure(figsize=(20, 10))
    plt.imshow(attention.numpy()[:len(labels_summary), 1:len(labels_original)])
    plt.xticks([i for i in range(len(labels_original)-1)], labels_original, rotation=75)
    plt.yticks([i for i in range(len(labels_summary))], labels_summary)
    plt.show()

In [ ]:
def summarize(text):
    with torch.no_grad():
        sequence = indices_from_text(text).unsqueeze(0)
        sequence_length = sequence.size(1)
        encoder_outputs, encoder_hidden = encoder(sequence.transpose(0, 1))
        
        decoder_input = torch.cuda.LongTensor([indices_from_text(TEXT.init_token)])
        hidden = encoder_hidden[:decoder.n_layers]
        summary_words = ['<sos>']
        max_summary_length = int(sequence_length * 0.25)
        decoder_attentions = torch.zeros(max_summary_length, sequence_length)
        
        for idx in range(max_summary_length):
            output, hidden, decoder_attention = decoder(
                decoder_input, 
                hidden, 
                encoder_outputs,
            )
            decoder_attentions[idx, :decoder_attention.size(2)] += decoder_attention.squeeze(0).squeeze(0).cpu().data
            top_v, top_i = output.data.topk(1)
            ni = top_i[0]
            if ni == indices_from_text(TEXT.eos_token):
                break
            else:
                summary_words.append(text_from_indices(ni))
            
            decoder_input = torch.cuda.LongTensor([ni])
        summary_words.append(TEXT.eos_token)
        summary = " ".join(summary_words).lstrip()
        return summary, decoder_attentions

In [ ]:
epochs = 10
lr = 0.01
grad_clip = 10.0
scheduler_step_size = 50
scheduler_gamma = 0.75

In [ ]:
hidden_size = 256
embed_size = 128

In [ ]:
print(f'[!] preparing dataset...')
text_size, summary_size = len(TEXT.vocab), len(SUMMARY.vocab)
print(f'[TRAIN]: {len(train_iter)} | {len(train_iter.dataset)}\t [TEST]: {len(test_iter)} | {len(test_iter.dataset)}')
print(f'[TEXT_vocab] & [SUMMARY_vocab] (same) {text_size}')

In [ ]:
print("[!] Instantiating models...")
encoder = EncoderRNN(text_size, embed_size, hidden_size,
                  n_layers=2, dropout=0.5)
decoder = DecoderRNN(embed_size, hidden_size, text_size,
                  n_layers=1, dropout=0.5)
seq2seq = Seq2Seq(encoder, decoder).cuda()
optimizer = optim.Adam(seq2seq.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=scheduler_step_size, gamma=scheduler_gamma)
print(seq2seq)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
best_val_loss = None
for e in notebook.tqdm(range(1, epochs+1)):
    train(e, seq2seq, optimizer, scheduler, train_iter, text_size, grad_clip, TEXT)
    val_loss = evaluate(seq2seq, val_iter, text_size, TEXT)
    print(f'[Epoch: {e}] val_loss: {val_loss} | val_pp: {math.exp(val_loss)}')

    # Save the model if the validation loss is the best we've seen so far.
    if not best_val_loss or val_loss < best_val_loss:
        print("[!] saving model...")
        if not os.path.isdir(".save"):
            os.makedirs(".save")
        torch.save(seq2seq.state_dict(), './.save/seq2seq_%d.pt' % (e))
        best_val_loss = val_loss
test_loss = evaluate(seq2seq, test_iter, text_size, TEXT)
print(f'[TEST] loss: {test_loss}')

In [ ]:
glob('../working/*')

In [ ]:
import os
os.chdir(r'../working')

In [ ]:
torch.save(seq2seq.state_dict(), 'pl_seq2seq_small.pt')

In [ ]:
eval_seq2seq = Seq2Seq(encoder, decoder).cuda()

In [ ]:
eval_seq2seq.load_state_dict(torch.load('../working/pl_seq2seq.pt'))

In [ ]:
eval_seq2seq.decoder

In [ ]:
def summarize2(text):
    with torch.no_grad():
        sequence = indices_from_text(text).unsqueeze(0)
        sequence_length = sequence.size(1)
        encoder_outputs, encoder_hidden = eval_seq2seq.encoder(sequence.transpose(0, 1))
        
        decoder_input = torch.LongTensor([indices_from_text(TEXT.init_token)]).cuda()
        hidden = encoder_hidden[:eval_seq2seq.decoder.n_layers]
        summary_words = ['<sos>']
        max_summary_length = int(sequence_length * 0.25)
        decoder_attentions = torch.zeros(max_summary_length, sequence_length)
        
        for idx in range(max_summary_length):
            output, hidden, decoder_attention = eval_seq2seq.decoder(
                decoder_input, 
                hidden, 
                encoder_outputs,
            )
            decoder_attentions[idx, :decoder_attention.size(2)] += decoder_attention.squeeze(0).squeeze(0).cpu().data
            top_v, top_i = output.data.topk(1)
            ni = top_i[0]
            if ni == indices_from_text(TEXT.eos_token):
                break
            else:
                summary_words.append(text_from_indices(ni))
            
            decoder_input = torch.LongTensor([ni]).cuda()
        summary_words.append(TEXT.eos_token)
        summary = " ".join(summary_words).lstrip()
        return summary, decoder_attentions

In [ ]:
for name, param in seq2seq.named_parameters():
    if param.requires_grad:
        print (name, param.data.shape)

In [ ]:
!pip install torchsummary
from torchsummary import summary

In [ ]:
summary

In [ ]:
summary(seq2seq.encoder, (500, 8), (100, 8))

In [ ]:
next(iter(train_iter)).text[0].shape, next(iter(train_iter)).lead[0].shape

In [ ]:
for name, param in seq2seq.named_parameters():
    if param.requires_grad:
        print (name, param.data.shape)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

Hidden size 128

In [ ]:
count_parameters(encoder)

In [ ]:
count_parameters(decoder)

In [ ]:
count_parameters(seq2seq)

Hidden size 256

In [ ]:
count_parameters(encoder)

In [ ]:
count_parameters(decoder)

In [ ]:
count_parameters(seq2seq)